In [23]:
import csv
import requests
from unidecode import unidecode 
import os

In [24]:
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'if-none-match': 'W/"7dd2d62cd7"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Chromium";v="118", "Opera GX";v="104", "Not=A?Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 OPR/104.0.0.0',

}
headers['If-Modified-Since'] = 'Sun, 3 Dec 2023 00:00:00 GMT'

In [25]:
players_details_csv_file_path = os.path.join('..', '..', 'data', 'sofascore', 'players_details_sofascore.csv')
players_ids_src = os.path.join('..', '..', 'data', 'players', 'players_ids.csv')
selected_keys = ["id", "slug", "team", "height", "preferredFoot", "marketValue"]

In [26]:
players_list = []
with open(players_ids_src, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        slug = row['slug']
        id_sofascore = row['id_sofascore']
        player_id = {'slug': slug,'id_sofascore': id_sofascore}
        players_list.append(player_id)

In [27]:
with open(players_details_csv_file_path, "w", newline="") as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=selected_keys)
    csv_writer.writeheader()

    # Zapisywanie danych do pliku CSV
    for player in players_list:
        response = requests.get(f'https://api.sofascore.com/api/v1/player/{player["id_sofascore"]}', headers=headers)
        player_response = response.json()
        player_data = player_response.get("player", {})
        if player["slug"] == player_data.get('slug', ""):
            marketValue = [str(player_data.get("proposedMarketValueRaw", {}).get("value", "")), player_data.get("proposedMarketValueRaw", {}).get("currency", "")]
            selected_data = {
                "id": player_data.get("id", ""),
                "slug": unidecode(player_data.get("slug", "")),
                "team": unidecode(player_data.get("team", {}).get("slug","")),
                "height": player_data.get("height", ""),
                "preferredFoot": player_data.get("preferredFoot", ""),
                "marketValue": " ".join(marketValue)
        }
        csv_writer.writerow(selected_data)

print(f"Dane zostały zapisane do pliku CSV: {players_details_csv_file_path}")

Dane zostały zapisane do pliku CSV: ..\..\data\sofascore\players_details_sofascore.csv
